In [1]:
import statsapi
import pandas as pd
from pybaseball import statcast
from tqdm import tqdm
import time

In [2]:
data_dir = "./datasets/"

In [3]:
schedule_2023 = pd.DataFrame(statsapi.schedule(start_date="03/25/2023",end_date="12/31/2023"))
schedule_2024 = pd.DataFrame(statsapi.schedule(start_date="03/25/2024",end_date="12/31/2024"))

schedule = pd.concat([schedule_2023,schedule_2024])
schedule['game_date'] = pd.to_datetime(schedule['game_date'])
schedule = schedule[schedule['status'].str.contains('Final')|schedule['status'].str.contains('Completed')]
schedule = schedule[schedule["game_type"]!="S"]

In [4]:
schedule["season"] = schedule["game_date"].apply(lambda x: int(str(x)[:4]))

In [13]:
mlb_batting_dataframe = pd.DataFrame()
mlb_pitching_dataframe = pd.DataFrame()
player_id_name = pd.DataFrame()


for i,row in tqdm(schedule.drop_duplicates(subset=['game_id']).iterrows()): 
    
    

    boxscore_dict = statsapi.boxscore_data(row['game_id'], timecode=None)
    
    local_player_id_name = pd.DataFrame(boxscore_dict['playerInfo']).T
    
    away_batters = pd.DataFrame(boxscore_dict['awayBatters'])
    away_batters['team_id'] = row['away_id']
    home_batters = pd.DataFrame(boxscore_dict['homeBatters'])
    home_batters['team_id'] = row['home_id']

    local_batters = pd.concat([home_batters,away_batters])


    away_pitchers = pd.DataFrame(boxscore_dict['awayPitchers'])
    away_pitchers['team_id'] = row['away_id']
    home_pitchers = pd.DataFrame(boxscore_dict['homePitchers'])
    home_pitchers['team_id'] = row['home_id']
    
    local_pitchers = pd.concat([home_pitchers,away_pitchers])

    local_batters['game_date'] = row['game_date']
    local_batters['game_id'] = row['game_id']
    local_batters['season'] = row['season']
    
    
    local_pitchers['game_date'] = row['game_date']
    local_pitchers['game_id'] = row['game_id']
    local_pitchers['season'] = row['season']    
    
    mlb_batting_dataframe = pd.concat([mlb_batting_dataframe,local_batters])
    mlb_pitching_dataframe = pd.concat([mlb_pitching_dataframe,local_pitchers])
    player_id_name = pd.concat([player_id_name,local_player_id_name]).drop_duplicates()
    
    time.sleep(1)


55it [01:19,  1.45s/it]


In [14]:
mlb_pitching_dataframe = mlb_pitching_dataframe[~mlb_pitching_dataframe["h"].str.contains("H")]
mlb_batting_dataframe = mlb_batting_dataframe[~mlb_batting_dataframe["h"].str.contains("H")]

In [ ]:
mlb_batting_dataframe.to_csv(data_dir+"mlb_batting_dataframe.csv",index=False)
mlb_pitching_dataframe.to_csv(data_dir+"mlb_pitching_dataframe.csv",index=False)
player_id_name.to_csv(data_dir+"player_id_name.csv",index=False)

In [ ]:
pitchDF_2023 = statcast(start_dt="2023-03-30", end_dt="2023-12-30")
pitchDF_2024 = statcast(start_dt="2024-03-30", end_dt="2024-12-30")

In [ ]:
pitchDF = pd.concat([pitchDF_2023,pitchDF_2024])

In [ ]:
pitchDF = pitchDF[['game_date',"game_pk",'batter', 'pitcher','events',
       'description', 'zone','pitch_type',  'balls', 'strikes','game_year','on_3b', 'on_2b', 'on_1b',
        'outs_when_up', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed','estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'woba_value', 'woba_denom', 'babip_value', 'iso_value',
       'launch_speed_angle', 'at_bat_number', 'pitch_number',]]

In [ ]:
pitchDF.to_csv(data_dir+"pitchDF.csv",index=False)